In [ ]:
from pyincore import IncoreClient, FragilityService, MappingSet
from pyincore.analyses.epfrepaircost import EpfRepairCost
from pyincore.analyses.epfdamage import EpfDamage
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability
import pyincore.globals as pyglobals

In [ ]:
client = IncoreClient()

In [ ]:
epf_id = "5d263f08b9219cf93c056c68"

### Electric Power Facility Damage

In [ ]:
epf_dmg = EpfDamage(client)
epf_dmg.load_remote_input_dataset("epfs", epf_id)

fragility_service = FragilityService(client)
# OSU Seaside Earthquake electric power facility mappings
mapping_set = MappingSet(fragility_service.get_mapping("5d489aa1b9219c0689f1988e"))  

epf_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

epf_dmg.set_parameter("result_name", "seaside_epf_dmg_result")
epf_dmg.set_parameter("hazard_type", "earthquake")

# Seaside Probabilistic EQ 5000yr
epf_dmg.set_parameter("hazard_id", "55dfbca0cb9219c101fd8a58d") 
epf_dmg.set_parameter("num_cpu", 4)
epf_dmg.run_analysis()

In [ ]:
epf_dmg_result = epf_dmg.get_output_dataset("result")
epf_dmg_result.get_dataframe_from_csv().head()

### Monte Carlo Simulation

In [ ]:
mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", epf_dmg_result)
mc.set_parameter("num_cpu", 4)
mc.set_parameter("num_samples", 100)
mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3", "DS_4"])
mc.set_parameter("failure_state_keys", ["DS_1", "DS_2", "DS_3", "DS_4"])

mc.set_parameter("result_name", "seaside_epf_mc") # name of csv file with results
mc.run()

In [ ]:
epf_sample_damage_states_result = mc.get_output_dataset("sample_damage_states")
epf_sample_damage_states_result.get_dataframe_from_csv().head()

### Electric Power Facility Repair Cost

In [ ]:
epf_repair_cost = EpfRepairCost(client)

# Seaside EPF
epf_repair_cost.load_remote_input_dataset("epfs", "5d263f08b9219cf93c056c68")

epf_repair_cost.load_remote_input_dataset("replacement_cost", "647dff5b4dd25160127ca192")

# can be chained with MCS
epf_repair_cost.set_input_dataset("sample_damage_states", epf_sample_damage_states_result)

# dmg ratios
epf_repair_cost.load_remote_input_dataset("epf_dmg_ratios", "6483354b41181d20004efbd7")

epf_repair_cost.set_parameter("result_name", "seaside_epf")
epf_repair_cost.set_parameter("num_cpu", 4)

# Run Analysis
epf_repair_cost.run_analysis()

In [ ]:
epf_repair_cost.get_output_dataset("result").get_dataframe_from_csv().head()